<a href="https://colab.research.google.com/github/dkaizhang/interpolate/blob/main/interpolate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np


gvc - this is where we get the implied vol for all underlyings and hedges.

hedge_bv - this is what I use to pull the implied vol for the hedge

For these two, I could create a table with all of the days and find the interpolated implied vol for the 22 day expiry. I think all I need are 'calculated_at' and 'interpolated vol'

rr - this is the realised for the underlying and hedge. the same row has both
I think this will not change in how I pull it. There will be different lookbacks.

scores - this is the source for the vol scores
For this I also should create a table with all of the days in common with gvc and find the interpolated scores for the 22 day expiry.

In [ ]:
df1 = pd.DataFrame({
    'calculated_at': pd.to_datetime(['2024-01-01', '2024-01-02']),
    'synthetic_expiry': pd.to_datetime(['2024-02-01','2024-02-02'])
})

df2 = pd.DataFrame({
    'calculated_at': pd.to_datetime(['2024-01-01', '2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02',  '2024-01-02']),
    'expiry': pd.to_datetime(['2024-01-28', '2024-01-29', '2024-01-30', '2024-01-28', '2024-02-05','2024-03-05']),
    'value': [100, 100, 101, 100, 200, 10000]
})

In [ ]:
# Function to interpolate value based on the closest dates
def interpolate_value(row, df2):
    # Filter df2 by the same calculated_at
    df2_filtered = df2[df2['calculated_at'] == row['calculated_at']]

    # Sort the filtered df2 by expiry
    df2_sorted = df2_filtered.sort_values('expiry')

    # Find the two closest expiry dates
    expiry_dates = df2_sorted['expiry']
    values = df2_sorted['value']

    # If there are less than 2 points, interpolation is not possible
    if len(expiry_dates) < 2:
        return np.nan

    # Perform interpolation
    interp_func = interp1d(expiry_dates.astype(np.int64), values, fill_value='extrapolate')
    interpolated_value = interp_func(row['synthetic_expiry'].value)

    return interpolated_value

In [ ]:
# Apply the interpolation function to each row in df1
df1['interpolated_value'] = df1.apply(interpolate_value, axis=1, df2=df2)

# Display the resulting dataframe
print(df1)

  calculated_at synthetic_expiry interpolated_value
0    2024-01-01       2024-02-01              103.0
1    2024-01-02       2024-02-02              162.5
